In [1]:
from tensorflow.python.framework.ops import disable_eager_execution
from sklearn.model_selection import train_test_split
disable_eager_execution()
import nltk
import re
import pandas as pd

In [2]:
df = pd.read_csv('D:/tesis/scisumm/scisumm.csv')
# df.head()

In [3]:
X = df['text']
y = df['summary']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75)

In [4]:
# printing out train and test sets
# print('X_train : ')
# print(X_train.head())
# print(X_train.shape)
# print('')
# print('X_test : ')
# print(X_test.head())
# print(X_test.shape)
# print('')
# print('y_train : ')
# print(y_train.head())
# print(y_train.shape)
# print('')
# print('y_test : ')
# print(y_test.head())
# print(y_test.shape)

In [5]:
X_train = X_train.str.lower()
X_test  = X_test.str.lower()

In [6]:
X_train.shape
# X_test.shape

(756,)

In [7]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
train   = [tokenizer.tokenize(s.lower()) for s in X_train]
test    = [tokenizer.tokenize(s.lower()) for s in X_test]

In [8]:
# Stopword Removal
from nltk.corpus import stopwords
nltk.download('stopwords')
listStopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Menghapus kata-kata di stopwords
train_stopwords = []
for sent in train:
  filter = [s for s in sent if s not in listStopword]
  train_stopwords.append(filter)
# train_stopwords

test_stopwords = []
for sent in test:
  filter = [s for s in sent if s not in listStopword]
  test_stopwords.append(filter)
# test_stopwords

# output merupakan sisa kata yg sudah dihapus

In [10]:
train   = [' '.join(t) for t in train_stopwords]
test    = [' '.join(t) for t in test_stopwords]

In [11]:
# importing modules
from nltk.stem import PorterStemmer
ps = PorterStemmer()

train   = [ps.stem(w) for w in train]
test    = [ps.stem(w) for w in test]

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()

train   = vec.fit_transform(train)
test    = vec.transform(test)

train   = train.toarray()
test    = test.toarray()

In [13]:
train.shape

(756, 60704)

In [14]:
test.shape

(253, 60704)

In [15]:
train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01000668, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.00778281, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00396712, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [17]:
# === HYPERMETER TUNING ===

import tensorflow as tf
from keras.layers import Input, Dense, Lambda
from keras.models import Sequential
from keras import backend as K
from keras import Model
from sklearn.model_selection import GridSearchCV

# Setup the network parameters:
original_dim = train.shape[1]
input_shape = (original_dim, )
latent_dim = 0

# build encoder model
def define_model(start_dim=0, intermediate_dim=0, latent_dim=0):
    model = Sequential()
    inputs      = model.add(Input(shape=input_shape, name='encoder_input'))
    x           = model.add(Dense(start_dim, activation='relu')(inputs))
    x           = model.add(Dense(intermediate_dim, activation='relu')(x))
    z_mean      = model.add(Dense(latent_dim, name='z_mean')(x))
    z_log_var   = model.add(Dense(latent_dim, name='z_log_var')(x))

    def sampling(args):
        z_mean, z_log_var = args
        batch   = K.shape(z_mean)[0]
        dim     = K.int_shape(z_mean)[1]
        # by default, random_normal has mean = 0 and std = 1.0
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Instantiate the encoder model:
    encoder = Model(inputs, z_mean, name='encoder')
    encoder.summary()

    # Build the decoder model:
    # def define_deccoder(start_dim=0, intermediate_dim=0, latent_dim=0):
    model = Sequential()
    latent_inputs   = model.add(Input(shape=(latent_dim,), name='z_sampling'))
    x               = model.add(Dense(intermediate_dim, activation='relu')(latent_inputs))
    x               = model.add(Dense(start_dim, activation='relu')(x))
    outputs         = model.add(Dense(original_dim, activation='sigmoid')(x))

    # Instantiate the decoder model:
    decoder = Model(latent_inputs, outputs, name='decoder')
    decoder.summary()

    # Instantiate the VAE model:
    outputs = decoder(encoder(inputs))
    vae     = Model(inputs, outputs, name='vae_mlp')
    vae.summary()

    # As in the Keras tutorial, we define a custom loss function:
    loss_object = tf.keras.losses.BinaryCrossentropy()
    def vae_loss(x, x_decoded_mean):
        xent_loss   = loss_object(x, x_decoded_mean)
        kl_loss     = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return xent_loss + kl_loss

    # We compile the model:
    vae.compile(optimizer='rmsprop', loss=vae_loss)
    return vae

from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=define_model, verbose=1)

# model = vae(build_fn=define_model, verbose=1)

# start_dim           = [200, 250, 500, 1000]
# intermediate_dim    = [100, 125, 150, 500]
# latent_dim          = [2,4,8,10,12,14,16,18,20]
# batch_size          = [100, 150, 200, 250]
# epochs              = [10,20,30,40,50]

start_dim           = [200]
intermediate_dim    = [100]
latent_dim          = [2]
batch_size          = [100]
epochs              = [10]

param_grid = dict(
    start_dim=start_dim,
    intermediate_dim=intermediate_dim,
    latent_dim=latent_dim,
    batch_size=batch_size,
    epochs = epochs
)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(train, train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean = %f (std=%f) with: %r" % (mean, stdev, param))


# Finally, we train the model:

results = model.fit(train, train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(test, test)
    )

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3644\1079192181.py:68: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=define_model, verbose=1)


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\wrappers\scikit_learn.py", line 239, in fit
    y = np.array(y)
numpy.core._exceptions._ArrayMemoryError: Unable to allocate 280. MiB for an array with shape (604, 60704) and data type float64

--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "c:\U

In [ ]:
import tensorflow as tf
from keras.layers import Input, Dense, Lambda
from keras import backend as K
from keras import Model

# Setup the network parameters:
original_dim = (72593)
input_shape = (original_dim, )
# intermediate_dim = 200
batch_size  = 16
latent_dim  = 250
epochs      = 10

# Map inputs to the latent distribution parameters:
# VAE model = encoder + decoder
# build encoder model
inputs      = Input(shape=input_shape, name='encoder_input')
# x           = Dense(intermediate_dim, activation='relu')(inputs)
x           = Dense(1000, activation='relu')(inputs)
x           = Dense(500, activation='relu')(x)
z_mean      = Dense(latent_dim, name='z_mean')(x)
z_log_var   = Dense(latent_dim, name='z_log_var')(x)

# Use those parameters to sample new points from the latent space:
# reparameterization trick
# instead of sampling from Q(z|X), sample epsilon = N(0,I)
# z = z_mean + sqrt(var) * epsilon
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch   = K.shape(z_mean)[0]
    dim     = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# Instantiate the encoder model:
encoder = Model(inputs, z_mean, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 72593)]           0         
                                                                 
 dense (Dense)               (None, 1000)              72594000  
                                                                 
 dense_1 (Dense)             (None, 500)               500500    
                                                                 
 z_mean (Dense)              (None, 250)               125250    
                                                                 
Total params: 73,219,750
Trainable params: 73,219,750
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Build the decoder model:
latent_inputs   = Input(shape=(latent_dim,), name='z_sampling')
# x               = Dense(intermediate_dim, activation='relu')(latent_inputs)
x               = Dense(500, activation='relu')(latent_inputs)
x               = Dense(1000, activation='relu')(x)
outputs         = Dense(original_dim, activation='sigmoid')(x)

# Instantiate the decoder model:
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_sampling (InputLayer)     [(None, 250)]             0         
                                                                 
 dense_2 (Dense)             (None, 500)               125500    
                                                                 
 dense_3 (Dense)             (None, 1000)              501000    
                                                                 
 dense_4 (Dense)             (None, 60704)             60764704  
                                                                 
Total params: 61,391,204
Trainable params: 61,391,204
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Instantiate the VAE model:
outputs = decoder(encoder(inputs))
vae     = Model(inputs, outputs, name='vae_mlp')
vae.summary()

# As in the Keras tutorial, we define a custom loss function:
loss_object = tf.keras.losses.BinaryCrossentropy()
def vae_loss(x, x_decoded_mean):
    xent_loss   = loss_object(x, x_decoded_mean)
    kl_loss     = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return xent_loss + kl_loss

Model: "vae_mlp"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 60704)]           0         
                                                                 
 encoder (Functional)        (None, 250)               61330750  
                                                                 
 decoder (Functional)        (None, 60704)             61391204  
                                                                 
Total params: 122,721,954
Trainable params: 122,721,954
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# We compile the model:
vae.compile(optimizer='rmsprop', loss=vae_loss)

# Finally, we train the model:

results = vae.fit(train, train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(test, test)
        )

Train on 756 samples, validate on 253 samples
Epoch 1/10
756/756 [==============================] - ETA: 0s - loss: 0.0761

c:\Users\LENOVO\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


756/756 [==============================] - 83s 110ms/sample - loss: 0.0761 - val_loss: 0.0036
Epoch 2/10
756/756 [==============================] - 67s 88ms/sample - loss: 0.0028 - val_loss: 0.0021
Epoch 3/10
756/756 [==============================] - 65s 87ms/sample - loss: 0.0021 - val_loss: 0.0020
Epoch 4/10
756/756 [==============================] - 70s 93ms/sample - loss: 0.0021 - val_loss: 0.0021
Epoch 5/10
756/756 [==============================] - 69s 92ms/sample - loss: 0.0020 - val_loss: 0.0019
Epoch 6/10
756/756 [==============================] - 64s 84ms/sample - loss: 0.0020 - val_loss: 0.0019
Epoch 7/10
756/756 [==============================] - 63s 84ms/sample - loss: 0.0020 - val_loss: 0.0019
Epoch 8/10
756/756 [==============================] - 62s 83ms/sample - loss: 0.0020 - val_loss: 0.0019
Epoch 9/10
756/756 [==============================] - 63s 83ms/sample - loss: 0.0020 - val_loss: 0.0019
Epoch 10/10
756/756 [==============================] - 63s 83ms/sample - l

In [ ]:
X_encoded = encoder.predict(train)
reconstruction = vae.predict(train)

print("\n")
print("Variational Autoencoder Encoder Reduced Scissum Dataset Size: : ", X_encoded.shape)
print("\n")
print("Variational Autoencoder Scissum Dataset Size: : ", reconstruction.shape)
print("\n")

c:\Users\LENOVO\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,




Variational Autoencoder Encoder Reduced Scissum Dataset Size: :  (756, 250)


Variational Autoencoder Scissum Dataset Size: :  (756, 60704)


